# ANÁLISE DE ATIVIDADES DE MANUTENÇÃO E DE CONFIABILIDADE DE ATIVOS DE UMA EMPRESA FABRICANTE DE SINALIZAÇÃO PARA VEÍCULOS PESADOS E COMERCIAIS

### Ana Luiza Figueiredo Gouvea

## Visão Geral (06-2021 a 12-2021)

Para a análise dos dados a partir de junho de 2021, estão sendo usadas planilhas geradas pelo novo software desenvolvido dentro da própria empresa. As planilhas são geradas no formato csv e xlsx e podem ser obtidas acessando o software dentro da rede interna. 

Diferente do software terceiro, o software interno da empresa gera planilhas separadas para os tipos de atividades classificadas como preventivas. 

Outro detalhe, é que o novo software trouxe possibilidades de outras classificações para as atividades, como a classificação "ajuste" e para atividades descritas como "corretivas" (ex: corretiva por erro de operação), permitindo análises mais delimitadas.


---

### Importação das bibliotecas 

In [ ]:
# Instalações de pacotes:
!pip install pyspark 
!pip install jupyter 
!pip install findspark 

#Serão necessários na parte de analise de analise de palavras -> Utilizado na parte de analise das atividades 'corretivas'.

In [ ]:
# -*- coding: utf-8 -*-
# Bibliotecas para analise dos dados:
import os
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as dt

# -----------------------------------------
#Instalação das bibliotecas para analise de palavras -> Utilizado na parte de analise das atividades 'corretivas':

import findspark
import pyspark
import nltk
import string
import collections
from collections import Counter
#---------------------------------------------
# Impressão das Versões das Bibliotecas utilizadas:
print(f'Matplot version: {matplotlib.__version__}')
print(f'Numpy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'Seaborn version: {sns.__version__}')
print(f'Findspark version: {findspark.__version__}')
print(f'Pyspark version: {pyspark.__version__}')
print(f'NLTK version: {nltk.__version__}')

#--------------------------------------------

print('Download das bibliotecas completo!')

### Importação dos arquivos

In [ ]:
# Importação do arquivo:
# Esse primeiro arquivo contém as informações de corretivas, ajustes e melhorias.
# Ambos os arquviso utilizam codificação UTF-8 e separação das colunas por ponto e virgula.
data = pd.read_csv(r"CAMINHO PARA ARQUIVO CSV",sep=';',encoding='utf-8')
data_prev = pd.read_excel(r"CAMINHO PARA ARQUIVO XLSX",sep=';',encoding='utf-8')

### Tratamento de Dados

#### Ajustes necessáriospara utilização do arquivo csv

Este arquivo possui informações de atividades classificadas como 'Corretivas', 'Ajuste' e 'Melhoria'.

In [ ]:
print(data)

In [ ]:
# Informações do DataFrame:
data.info()

In [ ]:
# Visualização das primeiras linhas do DataFrame:
data.head()

In [ ]:
# Não foi possível a utilização dos nomes das colunas por falha no reconhecimento:
#arquivo_fianl = data[['  AtivoLocal  ']]

In [ ]:
# Necessário download do arquivo sem o cabeçalho:
df = pd.read_csv(r"CAMINHO PARA ARQUIVO CSV",sep=';',encoding='utf-8',header=None)

In [ ]:
# Visualização das informações contidas no DataFrame:
df.info()

In [ ]:
# Primeiras linhas do DataFrame df:
df.head()

Vamos filtrar apenas as colunas que iremos utilizar em nossas análise, sendo elas: 'AtivoLocal', 'Criticidade','Tipo de Serviço','Tipo de Os','Descrição Ativo Local','Data/Hora Atividade','Atividade'.

In [ ]:
# Criação do primeiro arquivo com as colunas que iremos utilizar:
arquivo_final = df[[1,4,5,10,11,26,27]] # descrição dos numeros da colunas utilizadas
arquivo_final.columns = ['Ativo', 'Criticidade','Tipo Servico','Tipo OS','Descricao','Data','Atividade'] # nomeando as colunas
arquivo_final = arquivo_final # salvando o arquivo final com o mesmo nome

In [ ]:
# Mostrar informações do arquivo que será utilizado
arquivo_final.info()

# ** Análise do Período **

### Tipos de tarefas mais executadas

In [ ]:
#Foi criado uma coluna com o valor 1 para auxiliar nas contagens:
arquivo_final.insert(loc=0,column='contador',value=1)

In [ ]:
#Apagando linha que era cabeçalho anterior
arquivo_final.drop(index=0, inplace=True)
arquivo_final

In [ ]:
# Tipos de Tarefa:
tipos_tarefas_mais_realizadas = arquivo_final[['Tipo OS','contador']].groupby('Tipo OS').sum()
tipos_tarefas_mais_realizadas = tipos_tarefas_mais_realizadas.sort_values(by=['contador'],ascending=False)
tipos_tarefas_mais_realizadas

In [ ]:
#Grafico Tipos de Tarefa:
tipos_tarefas_mais_realizadas = tipos_tarefas_mais_realizadas.head(8)
indice_x = tipos_tarefas_mais_realizadas['contador']
nomes_tarefas = tipos_tarefas_mais_realizadas.index
plt.figure(figsize=(15,10))
sns.set_style('white')
figura_tarefas = sns.barplot(x=indice_x,y=nomes_tarefas,palette = 'inferno')
figura_tarefas.set_title('Os 10 tipos de Tarefas mais realizadas no período',fontsize = 20)
figura_tarefas.set_xlabel('Numero de atividades',fontsize=15)
figura_tarefas.set_ylabel('Tipos de Tarefas',fontsize = 15)
figura_tarefas

### Instalações com maior numero de atividades realizadas

Análise de dados de aividades realizadas em instalações (infraestrutura). A planilha baixada do Novo software não possui um campo para a distinção entre máquinas e infraestrutura, assim foi necessário procurar pelos respectivos nomes dados ás áreas.

In [ ]:
# Selecionar 'ativos' classificados como 'Instalações':
m2 = arquivo_final.loc[arquivo_final['Ativo'].str.contains("Área|Serralheria|Ferramentaria|Planta 1|Planta 2",na = False)]
m2 = m2.loc[~m2['Ativo'].str.contains("Talha")]
m2

In [ ]:
# Atividades para 'ativos' classificados como 'instalações':
instalações_atividades_mais_realizadas = m2[['Ativo','contador']].groupby('Ativo').sum()
instalações_atividades_mais_realizadas = instalações_atividades_mais_realizadas.sort_values(by=['contador'],ascending=False)
instalações_mais_realizadas = instalações_atividades_mais_realizadas.head(20)
instalações_mais_realizadas.head(10)

In [ ]:
# Grafico instalações:

instalações_mais = instalações_mais_realizadas.head(10)
indice_x = instalações_mais['contador']
nomes_instalações = instalações_mais.index
fig = plt.figure(figsize=(15,10))
sns.set_style('white')
figura_instalações = sns.barplot(x=indice_x,y=nomes_instalações ,palette = 'Purples_r')
figura_instalações.set_title('As 10 instalações de infraestrutura com o maior numero de atividades realizadas ',fontsize = 20)
figura_instalações.set_xlabel('Numero de atividades',fontsize=15)
figura_instalações.set_ylabel('Instalações',fontsize = 15)
figura_instalações 

### Instalações com menor numero de atividades realizadas

In [ ]:
# Atividades para 'ativos' classificados como 'instalações':
instalações_menos_realizadas = instalações_atividades_mais_realizadas.tail(10)
instalações_menos_realizadas

### Equipamentos com maior numero de atividades realizadas

Análise de dados de aividades realizadas em equipamentos (máquinas). A planilha baixada do Novo software não possui um campo para a distinção entre máquinas e infraestrutura, assim foi necessário procurar pelos respectivos nomes dados aos equipamentos. A inserção do nome 'Prensa' permite a procura de equipamentos denominados 'Prensa', 'Prensa Terminal' e 'Prensa Terminal automática' que são três tipos distintos de máquinas.

In [ ]:
# Selecionar ativos classificados como 'Equipamentos':
m1 = arquivo_final.loc[arquivo_final['Ativo'].str.contains("Prensa|Injetora|INJETORA|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
m1 = m1.loc[~m1['Ativo'].str.contains("Área")]

In [ ]:
# Selecionar ativos classificados como equipamentos que possuem o maior numero de atividades realizadas cadastradas:
equipamentos_atividades_mais_realizadas = m1[['Ativo','Descricao','contador']].groupby(['Ativo','Descricao']).sum()
equipamentos_atividades_mais_realizadas = equipamentos_atividades_mais_realizadas.sort_values(by=['contador'],ascending=False)
equipamentos_mais_realizados = equipamentos_atividades_mais_realizadas.head(20)
equipamentos_mais_realizados.head(10)

In [ ]:
# Grafico Equipamentos:

equipamentos_mais = equipamentos_mais_realizados.head(10)
indice_x = equipamentos_mais ['contador']
nomes_equipamentos = equipamentos_mais.index
fig = plt.figure(figsize=(15,10))
sns.set_style('white')
figura_equipamentos = sns.barplot(x=indice_x,y=nomes_equipamentos ,palette = 'bone')
figura_equipamentos.set_title('Os 10 Equipamentos com maior numero de atividades realizadas',fontsize = 20)
figura_equipamentos.set_xlabel('Numero de atividades',fontsize=15)
figura_equipamentos.set_ylabel('Equipamentos',fontsize = 15)
figura_equipamentos 

### Equipamentos com menor numero de atividades realizadas

In [ ]:
# Selecionar ativos classificados como equipamentos que possuem o menor numero de atividades realizadas cadastradas:
equipamentos_menos_realizados = equipamentos_atividades_mais_realizadas.tail(30)
equipamentos_menos_realizados

---

## -----------  Preventivas (05-2021 a 12-2021) ----------- 

### Tratamento de Dados

In [ ]:
# Criação do arquivo apenas com as colunas que serão utilizadas:
arquivo_final_prev = data_prev[['DataHora','NomeEquipamento','SetorEquip','Filial','TipoAtiv']]
arquivo_final_prev = arquivo_final_prev
arquivo_final_prev

In [ ]:
#Foi criado uma coluna numerada para auxiliar nas contagens:
arquivo_final_prev.insert(loc=0,column='contador',value=1)
arquivo_final_prev

### Instalações com maior numero de atividades preventivas realizadas

In [ ]:
# Selecionar os ativos classificados como infraestrutura com maior numero de atividades preventivas realizadas:
ativos_preventivas_mais_realizadas = arquivo_final_prev.loc[~arquivo_final_prev['NomeEquipamento'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller|Vaso",na = False)]
ativos_preventivas_mais_realizadas = ativos_preventivas_mais_realizadas[['NomeEquipamento','Filial','contador']].groupby(['NomeEquipamento','Filial']).sum()
ativos_preventivas_mais_realizadas  = ativos_preventivas_mais_realizadas .sort_values(by=['contador'],ascending=False)
preventivas_mais_realizadas = ativos_preventivas_mais_realizadas.head(10)
preventivas_mais_realizadas

### Instalações com menor numero de atividades preventivas realizadas

In [ ]:
# Selecionar os 10 ativos com menor numero de atividades preventivas realizadas:
preventivas_menos_realizadas = ativos_preventivas_mais_realizadas.tail(10)
preventivas_menos_realizadas

In [ ]:
# Mostrar informações das linhas para o ativo com a string 'Planta 1':
arquivo_final_prev.loc[arquivo_final_prev['NomeEquipamento'].str.contains("Planta 1",na = False)]

### Equipamentos com maior numero de atividades Preventivas cadastradas:

In [ ]:
# Selecionar ativos classificados como 'Equipamentos':
mp = arquivo_final_prev.loc[arquivo_final_prev['NomeEquipamento'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
mp = mp.loc[~mp['NomeEquipamento'].str.contains("Área")]
mp = mp.loc[~mp['NomeEquipamento'].str.contains("Vaso")]

# 'Equipamentos' com maior numero de atividades preventivas :
equipamentos_prev_mais_realizadas = mp[['NomeEquipamento','Filial','contador']].groupby(['NomeEquipamento','Filial']).sum()
equipamentos_prev_mais_realizadas = equipamentos_prev_mais_realizadas.sort_values(by=['contador'],ascending=False)
equipamentos_prev_mais_realizadas.head(10)


### Equipamentos com menor numero de atividades Preventivas cadastradas:

In [ ]:
# 'Equipamentos' com menor numero de atividades preventivas :
equipamentos_prev_menos_realizadas = equipamentos_prev_mais_realizadas.tail(10)
equipamentos_prev_menos_realizadas

### Atividades Preventivas X Mês X Ano

In [ ]:
# Visualização das datas com maior numero de atividades preventivas no período:
equip_preventivas = arquivo_final_prev
equip_preventivas['DataHora'] = pd.to_datetime(equip_preventivas['DataHora'], format= '%Y%m%d %H:%M:%S') 
                      
# Selecionar apenas coluna 'DataHora':
equip_preventivas_m = equip_preventivas[['DataHora']]

# DataFrame com o numero de preventivas por mês
equip_preventivas_m = equip_preventivas_m.groupby([equip_preventivas_m['DataHora'].dt.year.rename('year'), equip_preventivas_m['DataHora'].dt.month.rename('month')]).agg({'count'})
print(equip_preventivas_m)
print("O numero de atividades preventivas totais foi de: ",equip_preventivas_m['DataHora'].sum())


---

## -----------  Corretivas ----------- 

### Instalações com maior numero de atividades corretivas realizadas

In [ ]:
# Distinção das classificações de 'Corretivas' encontradas no arquivo para ativos descritos como 'infraestrutura':
ic = m2.loc[m2['Tipo OS'].str.contains("Corretiva",na = False)]
ic = ic.loc[~ic['Ativo'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
infra_corretivas_mais_realizadas = ic[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
infra_corretivas_mais_realizadas  = infra_corretivas_mais_realizadas.sort_values(by=['contador'],ascending=False)
infra_corretivas_mais_realizadas.head(10)

### Equipamentos com maior numero de atividades corretivas realizadas

In [ ]:
# Distinção das classificações de 'Corretivas' encontradas no arquivo para ativos descritos como 'equipamento':
ec = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Corretiva",na = False)]
ec = ec.loc[ec['Ativo'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
ec = ec.loc[~ec['Ativo'].str.contains("Área")]
# Visualização dos ativos classificados como 'equipamento 'com maior numero preventivas devido a falhas de funcionamento:
equi_corretivas_mais_realizadas = ec[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
equi_corretivas_mais_realizadas  = equi_corretivas_mais_realizadas .sort_values(by=['contador'],ascending=False)
equi_corretivas_mais_realizadas.head(10)

In [ ]:
# Visualização dos equipamentos com maior numero de atividades corretivas devido a erros operacionais:
ece = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Erro",na = False)]
ece = ece.loc[ece['Ativo'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
ece = ece.loc[~ece['Ativo'].str.contains("Área")]
equi_corretivas_mais_realizadas = ece[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
equi_corretivas_mais_realizadas  = equi_corretivas_mais_realizadas .sort_values(by=['contador'],ascending=False)
equi_corretivas_mais_realizadas.head(10)

In [ ]:
#visualização das informaçoes do DataFrame:
ece.info()

In [ ]:
#Criando um arquivo com o Tipo de Tarefa 'corretiva' para analise de palavras mais comuns, usada na parte de análise de falhas:
# Selecionando apenas 'Terefa'
corretiva_analise_tarefa = ece[['Atividade']]
corretiva_analise_tarefa.to_csv('NOME DO ARQUIVO CSV.csv',index=False)
corretiva_analise_tarefa.head()

### Atividades Corretivas X Mês X Ano

In [ ]:
# Visualização das datas com maior numero de atividades corretivas no período:
equip_corretivas = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Corretiva",na = False)]
equip_corretivas['Data'] = pd.to_datetime(equip_corretivas['Data'],infer_datetime_format=True) 
                    
equip_corretivas_m = equip_corretivas[['Data']]

# DataFrame com o numero de corretivas por mês no período:
equip_corretivas_m = equip_corretivas_m .groupby([equip_corretivas_m ['Data'].dt.year.rename('year'), equip_corretivas_m ['Data'].dt.month.rename('month')]).agg({'count'})
print(equip_corretivas_m)
print("O numero de atividades corretivas totais foi de: ",equip_corretivas_m['Data'].sum())

### Análise dos termos mais comuns para descrição das atividades realizadas

In [ ]:
# Habilitar findspark para uso do Spark
# Caminho para pacote instalado
findspark.init('CAMINHO EM QUE O PACOTE PYSPARK FOI SALVO')
# foi utilizado o' findspark.init' pois a função 'findspark.find()' não foi reconhecida.

In [ ]:
#Contagem de palavras com NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Download do arquivo que será utilizado.
# O arquivo foi criado durante as análises de dados do período:
arquivo_analise_falha = 'file:CAMINHO EM QUE O ARQUIVO CSV COM AS TAREFAS FOI SALVO'
stopwords = nltk.corpus.stopwords.words('portuguese') # Uso do idioma portugues
punctuation = string.punctuation

In [ ]:
# Abrindo arquivo csv baixado:
with open("NOME DO ARQUIVO CSV COM AS TAREFAS", encoding="utf8") as text_file:
    contents = text_file.read()

In [ ]:
from collections import Counter

palavras = contents.lower()
palavras = palavras.replace('\n',' ').replace('\t','').split(' ')

contador = Counter(palavras)
    
for i in contador.items():
    print (i)

In [ ]:
import collections

# Lista de palavras
palavras = contents.lower()
palavras = palavras.replace('\n',' ').replace('\t','').split(' ')

# Contador para as ocorrencias de cada palavra
contador = Counter(palavras)

# As 100 palavras mais frequentes
contador = contador.most_common(100)

print (f'{contador}')

### Média de Falhas Atendidas Por Dia Durante o Período Por Equipamento

Como não possuímos os tempos corretos de operação de cada equipamento, aqui foi analisado o intervalo de dias entre falhas.

In [ ]:
#Listas
lista_maquina = []

# Codigo da função que permitirá digitar os equipamentos que se deseja analisar
def menu():
    #Variável:
    continuar = 1
    #Código condicional para permitir adicionar os nomes das máquinas que se deseja analisar:
    while(continuar):
        continuar = int(input("0. Sair\n"+ "1. Para inserir maquinas\n"))            
        if(continuar == 1):  # se o nuero digitado for 1, a condição é atendida.
            maquina = input("Digite as maquinas para calculo da média: ")
            lista_maquina.append(maquina)
    print(lista_maquina)

# Para ativar o codigo da função
menu() 

In [ ]:
# Listas
dias =[]
dias_maquina = []
media_t = []
lista_m = []

#Variáveis
m = 0
x = 0
i = 0

# Código condicional

for maq in lista_maquina:
    equip_corretiva_mtbf = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Corretiva",na = False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Tipo Servico'].str.contains("FERRAMENTARIA",na =False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Criticidade'].str.contains("Molde",na = False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Tipo OS'].str.contains("Erro",na = False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[equip_corretiva_mtbf['Ativo'].str.contains(maq,na = False)]
    equip_corretiva_mtbf['Data'] = pd.to_datetime(equip_corretiva_mtbf['Data'],infer_datetime_format=True) 

    equip_corretiva_mtbf_tempo = equip_corretiva_mtbf[['Ativo','Descricao','Data']].groupby(['Ativo','Descricao'])
    equip_corretiva_mtbf_tempo = equip_corretiva_mtbf.sort_values(by=['Data'],ascending=False)


    m = equip_corretiva_mtbf_tempo['Data'].count()
    lista_m.append(m)
    x = m-1
        
    for i in range(x):
        # Data final
        d2 = equip_corretiva_mtbf_tempo['Data'].iloc[i]

        # Data inicial
        d1 = equip_corretiva_mtbf_tempo['Data'].iloc[i+1]

        # Calculo da quantidade de dias
        quantidade_dias = abs((d2 - d1).days)
        dias.append(quantidade_dias)
        i = i+1
        
           
    dias_maquina.append(dias)
    soma_dias = sum(dias)
    media_dias = soma_dias/x
    media_t.append(media_dias)    
    dias =[] # lista dias é novamente zerada para guardar apenas as informações da próxima maquina analisada
    
# DataFrame com os dados resultante
df_sr=pd.DataFrame({"Maquina":lista_maquina,"Ocorrências":lista_m,"Média":media_t})


#Utilizamos a função abs para garantir que a quantidade de dias de diferença seja sempre positiva, 
#independente da ordem em que as datas foram subtraídas

In [ ]:
df_sr.sort_values(by=['Ocorrências'],ascending=False)

In [ ]:
# Visualização das atividades corretivas realizadas nas injetoras da 'Planta 1':
# As Injetoras foram os equipamentos com **maior** numero de ocorrências no período.

# !Obs : Foram escolhidas injetoras que permaneceram na planta durante os períodos estudados.

# Listas
lista_injetoras = ['INJETORA 01','INJETORA 10','INJETORA 11','INJETORA 12','INJETORA 13','INJETORA 18','INJETORA 24','INJETORA 30']
dias =[]
dias_maquina = []
media_t = []
lista_m = []

#Variáveis
m = 0
x = 0
i = 0

# Código condicional

for inj in lista_injetoras:
    equip_corretiva_mtbf = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Corretiva",na = False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Tipo Servico'].str.contains("FERRAMENTARIA",na =False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Criticidade'].str.contains("Molde",na = False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Tipo OS'].str.contains("Erro",na = False)]
    equip_corretiva_mtbf = equip_corretiva_mtbf.loc[equip_corretiva_mtbf['Ativo'].str.contains(inj,na = False)]
    equip_corretiva_mtbf['Data'] = pd.to_datetime(equip_corretiva_mtbf['Data'],infer_datetime_format=True) 

    equip_corretiva_mtbf_tempo = equip_corretiva_mtbf[['Ativo','Descricao','Data']].groupby(['Ativo','Descricao'])
    equip_corretiva_mtbf_tempo = equip_corretiva_mtbf.sort_values(by=['Data'],ascending=False)


    m = equip_corretiva_mtbf_tempo['Data'].count()
    lista_m.append(m)
    x = m-1
        
    for i in range(x):
        # Data final
        d2 = equip_corretiva_mtbf_tempo['Data'].iloc[i]

        # Data inicial
        d1 = equip_corretiva_mtbf_tempo['Data'].iloc[i+1]

        # Calculo da quantidade de dias
        quantidade_dias = abs((d2 - d1).days)
        dias.append(quantidade_dias)
        i = i+1
        
    dias_maquina.append(dias)
    soma_dias = sum(dias)
    media_dias = soma_dias/x
    media_t.append(media_dias)    
    dias =[] # lista dias é novamente zerada para guardar apenas as informações da próxima maquina analisada
    
# DataFrame com os dados resultante
df_inj=pd.DataFrame({"Maquina":lista_injetoras,"Ocorrências":lista_m,"Média":media_t})

# Vizualização do DataFrame:
df_inj = df_inj.sort_values(by=['Ocorrências'],ascending=False)
df_inj

In [ ]:
# Visualização das atividades corretivas realizadas nas injetora 18 da 'Planta 1':
# A Injetora 18 foi o equipamento com **maior** numero de ocorrências no período.
equip_corretiva_mtbf = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Corretiva",na = False)]
equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Criticidade'].str.contains("Molde",na = False)]
equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Tipo OS'].str.contains("Erro",na = False)]
equip_corretiva_mtbf = equip_corretiva_mtbf.loc[~equip_corretiva_mtbf['Tipo Servico'].str.contains("FERRAMENTARIA",na =False)]
equip_corretiva_mtbf = equip_corretiva_mtbf.loc[equip_corretiva_mtbf['Ativo'].str.contains("INJETORA 18",na = False)]
equip_corretiva_mtbf['Data'] = pd.to_datetime(equip_corretiva_mtbf['Data'],infer_datetime_format=True) 
                    
equip_corretiva_mtbf_tempo = equip_corretiva_mtbf[['Ativo','Data']].groupby('Ativo')
equip_corretiva_mtbf_tempo = equip_corretiva_mtbf.sort_values(by=['Data'],ascending=False)
equip_corretiva_mtbf_tempo.head(10)

---

##  ----------- Preventivas x Corretivas  -----------

In [ ]:
#Criação de tabela com os dados dados anteriores de preventiva e corretiva
m = pd.merge(equip_preventivas_m, equip_corretivas_m , how = 'inner', on = 'month')
m.columns = ['Preventivas','Corretivas']
m

In [ ]:
# Grafico com o numero de atividades preventivas e corretivas
prev_b = m['Preventivas']
corret_b = m['Corretivas']
resultado = pd.crosstab(prev_b,corret_b)
indice = m.index
fig, ax = plt.subplots(figsize = [15,10])
x = np.arange(len(indice))
ax.set_xticks(x)
ax.set_xticklabels(indice)
ax.bar(x-0.2,prev_b,width = 0.4)
ax.bar(x+0.2,corret_b,width = 0.4)
# width é para alterar a largura da minha barra
# deslocando o x nós conseguimos colocar uma barra do lado da outra
plt.legend(['Preventivas','Corretivas'])

In [ ]:
# Craição de coluna com a soma total das atividades por mÊs e a porcentagem de cada atividade 
m.insert(loc=2,column='soma',value= m.sum(axis=1))
por_prev = m['Preventivas']/m['soma'] * 100
por_corret = m['Corretivas']/m['soma'] *100
m.insert(loc=3,column='% Prev',value= por_prev)
m.insert(loc=4,column='% Corret',value= por_corret)
m

In [ ]:
# Garfico comparativo %Preventivas x %Corretivas
y = m.index
prev = m['% Prev']
corret= m['% Corret']
plt.figure(figsize=(15, 10))
plt.plot(y,prev,label = '% Preventiva',linewidth=2)
plt.plot(y,corret,label = '% Corretiva',linewidth=2)
plt.legend(fontsize=10)

plt.title('Preventiva x Corretiva',fontsize=20)
plt.xlabel('Mês',fontsize=15)
plt.ylabel('Porcentagem',fontsize=15)


---

##  -----------  Melhorias  ----------- 

### Instalações com o maior numero de melhorias realizadas

In [ ]:
# Distinção das classificações de 'Melhoria' encontradas no arquivo para ativos descritos como 'instalação':
am = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Melhoria",na = False)]
am = am.loc[~am['Ativo'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller|Vaso",na = False)]
ativos_melhorias_mais_realizadas = am[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
ativos_melhorias_mais_realizadas  = ativos_melhorias_mais_realizadas .sort_values(by=['contador'],ascending=False)
ativos_melhorias_mais_realizadas.head(10)

### Equipamentos com o maior numero de melhorias realizadas

In [ ]:
# Equipamentos com maior numero de atividades de melhoria realizadas
em = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Melhoria",na = False)]
em = em.loc[em['Ativo'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
em = em.loc[~em['Ativo'].str.contains("Área")]

equi_melhorias_mais_realizadas = em[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
equi_melhorias_mais_realizadas  = equi_melhorias_mais_realizadas .sort_values(by=['contador'],ascending=False)
equi_melhorias_mais_realizadas.head(10)

---

##  -----------  Ajustes  ----------- 

### Instalações com maior numero de atividades classificadas como ajuste 

In [ ]:
# Infraestruturas com maior numero de atividades de ajuste realizadas
aca = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Ajuste",na = False)]
aca = aca.loc[~aca['Ativo'].str.contains("Prensa|INJETORA|Injetora|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller|Vaso",na = False)]
ativos_ajustes_mais_realizadas = aca[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
ativos_ajustes_mais_realizadas  = ativos_ajustes_mais_realizadas .sort_values(by=['contador'],ascending=False)
ativos_ajustes_mais_realizadas.head(10)

### Equipamentos com maior numero de atividades classificadas como ajuste 

In [ ]:
# Equipamentos com maior numero de atividades de ajuste realizadas
eca = arquivo_final.loc[arquivo_final['Tipo OS'].str.contains("Ajuste",na = False)]
eca = eca.loc[eca['Ativo'].str.contains("Prensa|Injetora|INJETORA|Ultrassom|Termo|Torre|Compressor|Estufa|Printer|Pick|Forno|Talha|Meta|chiller",na = False)]
eca = eca.loc[~eca['Ativo'].str.contains("Área")]
equi_ajustes_mais_realizadas = eca[['Ativo','Descricao','Tipo OS','contador']].groupby(['Ativo','Descricao','Tipo OS']).sum()
equi_ajustes_mais_realizadas = equi_ajustes_mais_realizadas.sort_values(by=['contador'],ascending=False)
equi_ajustes_mais_realizadas.head(10)

In [ ]:
#Criando um arquivo com o Tipo de Tarefa 'ajuste' para analise de palavras mais comuns, usada na parte de análise de falhas:
ajustes_analise_falha = eca[['Atividade']]
ajustes_analise_falha.to_csv('NOME DO ARQUIVO 2 CSV.csv',index=False)
ajustes_analise_falha.head()

### Análise dos termos mais comuns para descrição das atividades realizadas

In [ ]:
# Download do arquivo que será utilizado.
# O arquivo foi criado durante as análises de dados do período:
arquivo_analise_falha = 'file:CAMINHO EM QUE O ARQUIVO 2 CSV COM AS TAREFAS FOI SALVO'

# Abrindo arquivo csv baixado:
with open("NOME ARQUIVO 2 CSV", encoding="utf8") as text_file:
    contents_2 = text_file.read()

from collections import Counter #para encontrar os elementos de lista mais comuns

palavras_2 = contents_2.lower()
palavras_2 = palavras_2.replace('\n',' ').replace('\t','').split(' ')

# Contador para as ocorrencias de cada palavra
contador_2 = Counter(palavras_2)
    
for i in contador_2.items():
    print (i)

import collections #para ajudá-lo a trabalhar com tuplas, dicionários e listas

# Lista de palavras
palavras_2 = contents_2.lower()
palavras_2 = palavras_2.replace('\n',' ').replace('\t','').split(' ')


# As 100 palavras mais frequentes
contador_2 = contador_2.most_common(100)
print('\n\n')
print (f'Palavras que mais aparecem : {contador_2}')

---